In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2


2023-11-23 17:36:45.628867: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
interpreter = tf.lite.Interpreter(model_path="../models/3.tflite")
interpreter.allocate_tensors()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [20]:
def draw_key_points(frame, keypoints, conf_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))

    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > conf_threshold:
            cv2.circle(frame,(int(kx), int(ky)), 15, (0, 255, 0), 5)
    return frame


In [23]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()

    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192, 192)
    input_image = tf.cast(img, dtype=tf.float32)

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]["index"], np.array(input_image))
    interpreter.invoke()
    key_points_with_scores = interpreter.get_tensor(output_details[0]["index"])

    draw_key_points(frame,key_points_with_scores, conf_threshold=0.7)

    cv2.imshow("Live Feed Test", frame)

    if cv2.waitKey(10) & 0xFF==ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


-1